In [ ]:
!pip install scipy numpy six

In [ ]:
# If running in Google Colab, import files
try:
    import google.colab
    in_colab = True
except:
    in_colab = False

if in_colab:
    !git clone https://github.com/aiken516/one-pixel-attack-keras.git
    !mv -v one-pixel-attack-keras/* .
    !rm -rf one-pixel-attack-keras

# Python Libraries
%matplotlib inline
import pickle
import numpy as np
import pandas as pd
import matplotlib
from keras.datasets import cifar10
from keras import backend as K

# Custom Networks
from networks.lenet import LeNet
#from networks.pure_cnn import PureCnn
#from networks.network_in_network import NetworkInNetwork
from networks.resnet import ResNet
#from networks.densenet import DenseNet
#from networks.wide_resnet import WideResNet
#from networks.capsnet import CapsNet

# Helper functions
from differential_evolution import differential_evolution
import helper

import time

matplotlib.style.use('ggplot')
np.random.seed(100)

## Load Dataset

For this attack, we will use the [Cifar10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) packaged by Keras. The task of the dataset is to correctly classify a 32x32 pixel image in 1 of 10 categories (e.g., bird, deer, truck).

The code below will load the Cifar10 dataset. Keras will need to download the dataset if it is not cached locally already.

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
image_id = 99 # Image index in the test set
helper.plot_image(x_test[image_id])

In [5]:
def perturb_image(xs, img):
    # If this function is passed just one perturbation vector,
    # pack it in a list to keep the computation the same
    if xs.ndim < 2:
        xs = np.array([xs])

    # Copy the image n == len(xs) times so that we can
    # create n new perturbed images
    tile = [len(xs)] + [1]*(xs.ndim+1)
    imgs = np.tile(img, tile)

    # Make sure to floor the members of xs as int types
    xs = xs.astype(int)

    for x,img in zip(xs, imgs):
        # Split x into an array of 5-tuples (perturbation pixels)
        # i.e., [[x,y,r,g,b], ...]
        pixels = np.split(x, len(x) // 5)
        for pixel in pixels:
            # At each pixel's x,y position, assign its rgb value
            x_pos, y_pos, *rgb = pixel
            img[x_pos, y_pos] = rgb

    return imgs

In [ ]:
image_id = 99 # Image index in the test set
pixel = np.array([16, 16, 255, 255, 0]) # pixel = x,y,r,g,b
image_perturbed = perturb_image(pixel, x_test[image_id])[0]

helper.plot_image(image_perturbed)

In [ ]:
lenet = LeNet()
resnet = ResNet()

models = [lenet, resnet] #, resnet]

## Uncomment below to load more models to play with. Make sure the model files exist by training or downloading them.

# lenet = LeNet()
# pure_cnn = PureCnn()
# net_in_net = NetworkInNetwork()
# resnet = ResNet()
# densenet = DenseNet()
# wide_resnet = WideResNet()
# capsnet = CapsNet()

# models = [lenet, pure_cnn, net_in_net, resnet, densenet, wide_resnet, capsnet]

In [ ]:
network_stats, correct_imgs = helper.evaluate_models(models, x_test, y_test)
correct_imgs = pd.DataFrame(correct_imgs, columns=['name', 'img', 'label', 'confidence', 'pred'])
network_stats = pd.DataFrame(network_stats, columns=['name', 'accuracy', 'param_count'])

network_stats

In [7]:
def predict_classes(xs, img, target_class, model, minimize=True):
    # Perturb the image with the given pixel(s) x and get the prediction of the model
    imgs_perturbed = perturb_image(xs, img)
    predictions = model.predict(imgs_perturbed)[:,target_class]
    # This function should always be minimized, so return its complement if needed
    return predictions if minimize else 1 - predictions

In [ ]:
image_id = 384
pixel = np.array([16, 13,  25, 48, 156])
model = resnet

true_class = y_test[image_id, 0]
prior_confidence = model.predict_one(x_test[image_id])[true_class]
confidence = predict_classes(pixel, x_test[image_id], true_class, model)[0]

print('Confidence in true class', class_names[true_class], 'is', confidence)
print('Prior confidence was', prior_confidence)
helper.plot_image(perturb_image(pixel, x_test[image_id])[0])

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

# Axes의 타이틀을 설정
ax.set_title("Graph", size = 20)

# 축 라벨을 설정
ax.set_xlabel("x", size = 14)
ax.set_ylabel("y", size = 14)
ax.set_zlabel("confidence", size = 14)

In [ ]:
def one_color_graph(pixel, img_id, model, true_class, color):
    img = x_test[img_id]
    color_list = []
    confidence_list = []

    # 그래프의 타이틀을 설정
    plt.title("id:{0} model:{1} ({2}, {3})".format(img_id, model.name, pixel[0], pixel[1]), size = 10)


    # color 인자로 변경할 값 찾기
    if color == 'r':
        color_num = 2
    elif color == 'g':
        color_num = 3
    elif color == 'b':
        color_num = 4

    for rgb in range(0, 255, step):
        pixel[color_num] = rgb

        imgs_perturbed = perturb_image(pixel, img)
        color_list.append(rgb)
        predictions = model.predict_one(imgs_perturbed)[true_class]
        confidence_list.append(predictions)

    plt.plot(color_list, confidence_list, color)
    plt.show()

def two_color_graph(pixel, img_id, model, true_class, color_1, color_2):
    img = x_test[img_id]
    color_list_x = []
    color_list_y = []
    confidence_list = []

    if color_1 == 'r':
        color_num_1 = 2
    elif color_1 == 'g':
        color_num_1 = 3
    elif color_1 == 'b':
        color_num_1 = 4

    if color_2 == 'r':
        color_num_2 = 2
    elif color_2 == 'g':
        color_num_2 = 3
    elif color_2 == 'b':
        color_num_2 = 4

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Axes의 타이틀을 설정
    ax.set_title("id:{0} model:{1} ({2}, {3})".format(img_id, model.name, pixel[0], pixel[1]), size = 10)

    # 축 라벨을 설정
    ax.set_xlabel(color_1, size = 14)
    ax.set_ylabel(color_2, size = 14)
    ax.set_zlabel("confidence", size = 14)

    for rgb_x in range(0, 255, step):
      for rgb_y in range(0, 255, step):
          pixel[color_num_1] = rgb_x
          pixel[color_num_2] = rgb_y

          imgs_perturbed = perturb_image(pixel, img)
          color_list_x.append(rgb_x)
          color_list_y.append(rgb_y)
          predictions = model.predict_one(imgs_perturbed)[true_class]
          confidence_list.append(predictions)

    ax.scatter(color_list_x, color_list_y, confidence_list, 'r', alpha = 0.5)

def three_color_graph(pixel, img, model, true_class):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Axes의 타이틀을 설정
    ax.set_title("id:{0} model:{1} ({2}, {3})".format(img_id, model.name, pixel[0], pixel[1]), size = 10)

    # 축 라벨을 설정
    ax.set_xlabel("r", size = 14)
    ax.set_ylabel("g", size = 14)
    ax.set_zlabel("b", size = 14)

    for r in range(0, 255, step):
      pixel[2] = r
      for g in range(0, 255, step):
          pixel[3] = g
          for b in range(0, 255, step):
              pixel[4] = b

              imgs_perturbed = perturb_image(pixel, img)
              predictions = model.predict_one(imgs_perturbed)[true_class]
              ax.scatter(r, g, b, 'r', alpha = predictions)

In [ ]:
image_id = 384

pixel = np.array([13, 25,  92, 109,  76])

helper.plot_image(perturb_image(pixel, x_test[image_id]))

In [ ]:
image_id = 384
model = resnet

print(model.name)

In [ ]:
step = 1
image_id = 384
model = lenet
true_class = y_test[image_id, 0]
print(class_names[true_class])
helper.plot_image(x_test[image_id])

pixel = np.array([13, 25,  0, 0, 0])

In [ ]:
one_color_graph(pixel, image_id, model, true_class, 'r')
one_color_graph(pixel, image_id, model, true_class, 'g')
one_color_graph(pixel, image_id, model, true_class, 'b')

In [ ]:
step = 8
model = lenet
two_color_graph(pixel, image_id, model, true_class, 'r', 'g')
two_color_graph(pixel, image_id, model, true_class, 'r', 'b')
two_color_graph(pixel, image_id, model, true_class, 'g', 'b')